<a href="https://colab.research.google.com/github/Aravindh4404/FYPSeagullClassification01/blob/main/LATESTVGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The latest model train below That is /content/drive/My Drive/FYP/VGGModel/HQ3Optuna_20250218/latest_checkpoint.pth is not that good regarding the interpretability and the prediction number of images compared to original vgg model

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter
import optuna
from optuna.trial import TrialState

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Define the device for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define paths
data_path = '/content/drive/My Drive/FYP/Dataset/HQ3/train'
test_data_path = '/content/drive/My Drive/FYP/Dataset/HQ3/test'
checkpoint_folder = f'/content/drive/My Drive/FYP/VGGModel/HQ3Optuna_{datetime.now().strftime("%Y%m%d")}/'
os.makedirs(checkpoint_folder, exist_ok=True)

# Fixed checkpoint file path
checkpoint_file = os.path.join(checkpoint_folder, 'latest_checkpoint.pth')

# Calculate class weights
original_train_dataset = datasets.ImageFolder(data_path)
targets = [s[1] for s in original_train_dataset.samples]
class_counts = Counter(targets)
total = sum(class_counts.values())
class_weights = torch.tensor([
    total/(2*class_counts[0]),  # Smaller class
    total/(2*class_counts[1])   # Larger class
]).float().to(device)

# Data transformations
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.05, 0.05)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.RandomResizedCrop(224, scale=(0.95, 1.0)),
    transforms.RandomAdjustSharpness(1.2, p=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

transform_val_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Create base dataset
full_train_dataset = datasets.ImageFolder(data_path, transform=transform_train)
test_dataset = datasets.ImageFolder(test_data_path, transform=transform_val_test)

# Stratified train-val split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.05, random_state=42)
train_indices, val_indices = next(sss.split(np.zeros(len(targets)), targets))
train_dataset = Subset(full_train_dataset, train_indices)
val_dataset = Subset(full_train_dataset, val_indices)

# Modified VGG model with flexible classifier
class VGG16Modified(nn.Module):
    def __init__(self, dropout_rate, hidden_units):
        super().__init__()
        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        num_ftrs = self.vgg.classifier[6].in_features

        # Build custom classifier
        classifier_layers = []
        for units in hidden_units:
            classifier_layers.extend([
                nn.Linear(num_ftrs, units),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            num_ftrs = units
        classifier_layers.append(nn.Linear(num_ftrs, 2))

        self.vgg.classifier[6] = nn.Sequential(*classifier_layers)

    def forward(self, x):
        return self.vgg(x)

def save_checkpoint(model, optimizer, epoch, checkpoint_file):
    """Save model checkpoint."""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_file)
    print(f"Checkpoint saved at epoch {epoch+1} to {checkpoint_file}")

def load_checkpoint(model, optimizer, checkpoint_file):
    """Load model checkpoint."""
    if os.path.exists(checkpoint_file):
        checkpoint = torch.load(checkpoint_file)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Resuming training from epoch {start_epoch}")
        return start_epoch
    return 0

def train(model, train_loader, val_loader, optimizer, scheduler, epochs):
    """Training loop with checkpointing."""
    start_epoch = load_checkpoint(model, optimizer, checkpoint_file)

    for epoch in range(start_epoch, epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss(weight=class_weights)(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 2.0)
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                correct += (model(inputs).argmax(1) == labels).sum().item()
        val_acc = 100 * correct / len(val_dataset)
        scheduler.step(val_acc)

        # Save checkpoint after each epoch
        save_checkpoint(model, optimizer, epoch, checkpoint_file)

        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {running_loss/len(train_loader):.4f}, Val Acc: {val_acc:.2f}%")

# Final training function
def full_train(model, optimizer, scheduler, epochs=50):
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    train(model, train_loader, val_loader, optimizer, scheduler, epochs)

# Load best parameters from Optuna study
best_params = {
    'lr': 0.0001,
    'weight_decay': 0.001,
    'dropout_rate': 0.5,
    'hidden_units': [512],
    'optimizer': 'AdamW',
    'batch_size': 16,
    'grad_clip': 2.0,
    'sched_factor': 0.1,
    'sched_patience': 3
}

# Initialize final model
final_model = VGG16Modified(
    dropout_rate=best_params['dropout_rate'],
    hidden_units=best_params.get('hidden_units', [])
).to(device)

# Initialize optimizer and scheduler
optimizer = optim.AdamW(final_model.parameters(),
                       lr=best_params['lr'],
                       weight_decay=best_params['weight_decay'])

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=best_params['sched_factor'],
    patience=best_params['sched_patience']
)

# Execute final training
full_train(final_model, optimizer, scheduler, epochs=50)

# Test evaluation
def test(model):
    model.eval()
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            correct += (model(inputs).argmax(1) == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / len(test_dataset):.2f}%")

test(final_model)

Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checkpoint saved at epoch 1 to /content/drive/My Drive/FYP/VGGModel/HQ3Optuna_20250218/latest_checkpoint.pth
Epoch [1/50] - Loss: 0.6334, Val Acc: 94.74%
Checkpoint saved at epoch 2 to /content/drive/My Drive/FYP/VGGModel/HQ3Optuna_20250218/latest_checkpoint.pth
Epoch [2/50] - Loss: 0.7533, Val Acc: 86.84%
Checkpoint saved at epoch 3 to /content/drive/My Drive/FYP/VGGModel/HQ3Optuna_20250218/latest_checkpoint.pth
Epoch [3/50] - Loss: 0.4721, Val Acc: 97.37%
Checkpoint saved at epoch 4 to /content/drive/My Drive/FYP/VGGModel/HQ3Optuna_20250218/latest_checkpoint.pth
Epoch [4/50] - Loss: 0.2428, Val Acc: 94.74%
Checkpoint saved at epoch 5 to /content/drive/My Drive/FYP/VGGModel/HQ3Optuna_20250218/latest_checkpoint.pth
Epoch [5/50] - Loss: 0.1502, Val Acc: 94.74%
Checkpoint saved at epoch 6 to /content/drive/My Drive/FYP/VGGModel/HQ3Optuna_2025

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
